In [ ]:
import multiprocessing
import pandas as pd
import re

In [ ]:
tables = pd.read_csv('output-data/statistics/expanded_tables_annotations_cpa.csv')

In [ ]:
cpa_statistics = pd.read_csv('output-data/statistics/cpa_statistics.csv')
cpa_statistics = cpa_statistics.loc[(cpa_statistics['column_count'] >= 50) ]
cpa_statistics

In [ ]:
rels = pd.read_csv('data/Final CTA and CPA Labels.csv')
rels = rels.loc[rels['CPA label'].isin(cpa_statistics['cpa_label'].tolist())]
rels

In [ ]:
rel_lbls = {}
for index, row in tables.iterrows():
    for col in eval(row['rel_labels']):
        if eval(row['rel_labels'])[col] in cpa_statistics['cpa_label'].tolist():
            rel_lbls[col] = eval(row['rel_labels'])[col] 
del rel_lbls['name']

In [ ]:
selection = pd.read_csv('output-data/cpa-datasets/selected_1_2.csv')
selection['selected_cols_number'] = selection['selected_cols'].apply(lambda row: len(eval(row)))
selection_dict = selection.sort_values('selected_cols_number').to_dict('records')

selected_cols = {}
for row in selection_dict:
    selected_cols[row['file_name']] = eval(row['selected_cols'])

In [ ]:
#Select from already selected tables all non-annotated columns with a limit of 6500 columns
count_class = {} #Count columns per Schema.org type/class
rel_count = {} #Count columns per CPA label

for r in cpa_statistics['cpa_label'].tolist():
    if r != 'name':
        rel_count[r] = 0

for tab in selection_dict:
    sel = eval(tab['selected_cols'])
    rels = eval(tab['rel_labels'])

    if tab['class'] not in count_class:
        count_class[tab['class']] = 0

    #Select a maximum of 6500 columns per Schema.org type/class 
    #if count_class[tab['class']] < 6500:
    for col in eval(tab['all_cols']):
        if col in rel_lbls:
            #If column has not been yet selected:
            if col not in sel and eval(tab['all_cols'])[col] >= 70 and count_class[tab['class']] < 6500:  #: and rel_count[rels[col]] < 3000
                count_class[tab['class']] += 1
                #Add to selected
                selected_cols[tab['file_name']][col] = 'Random'
                rel_count[rels[col]] += 1

In [ ]:
#Mark only labels that have not yet reached at least 100 examples
low_labels = [ x for x in rel_count if rel_count[x] < 100 ]

In [ ]:
#Add more examples to CTA labels that have not reached enough examples
#count_class_2 = {} #Count columns per Schema.org type, do not pass 6500 columns

for cl in count_class:
    #count_class_2[cl] = 0
    if count_class[cl] < 6500:
        d = tables.loc[(~tables['file_name'].isin(selected_cols)) & (tables['class'] == cl ) & (tables['overall_table_density'] >= 70 )]
        
        for index, row in d.sort_values('column_count', ascending=False).iterrows():
            cpa = eval(row['rel_labels'])
            for col in eval(row['all_cols']):
                if eval(row['all_cols'])[col] >= 70 and col in rel_lbls:
                    if rel_count[cpa[col]] < 2000: #2000
                        rel_count[cpa[col]] += 1
                        if count_class[cl] < (6500 - count_class[cl]):
                            if row['file_name'] not in selected_cols:
                                selected_cols[row['file_name']] = {}
                            selected_cols[row['file_name']][col] = 'Random 2'
                            count_class[cl] += 1
                            
        

In [ ]:
# Select maximum 300 columns per CPA label
# new_cols = {}
# for l in low_labels:
#     new_cols[l] = []
    
for index, row in tables.sort_values('column_count', ascending=False).iterrows():
    rels = eval(row['rel_labels'])
    
    for col in eval(row['all_cols']):
        if col in rel_lbls and rels[col] in low_labels and rel_count[rels[col]] < 700:
                
            if (row['file_name'] not in selected_cols or col not in selected_cols[row['file_name']]) and eval(row['all_cols'])[col] >= 70:
                
                rel_count[rels[col]] += 1
                
                #add to selected columns
                if row['file_name'] not in selected_cols:
                    selected_cols[row['file_name']] = {}
                selected_cols[row['file_name']][col] = 'Random 3'
          

## Assemble all selected columns

In [ ]:
selection = tables.loc[ tables['file_name'].isin(selected_cols)]
sel_cols = []
for index, row in selection.iterrows():
    sel_cols.append(selected_cols[row['file_name']])
selection['selected_cols'] = sel_cols
selection

In [ ]:
res = []
for index, row in selection.iterrows():
    rel_labels = eval(row['rel_labels'])
    type_labels = eval(row['type_labels'])
    densities = eval(row['all_cols'])
    
    for cols in selected_cols[row['file_name']]:
        if cols in rel_labels and cols in type_labels:
            res.append([row['class'], cols, row['file_name'], rel_labels[cols], type_labels[cols], densities[cols], selected_cols[row['file_name']][cols] ])
        

In [ ]:
dataset = pd.DataFrame(res, columns=['class', 'column_name', 'file_name', 'relation_label', 'type_label', 'density', 'selection_type'])
dataset

In [ ]:
dataset = dataset.loc[dataset['density'] >= 10 ]
dataset

In [ ]:
#Remove columns belonging to CTA labels that do not have a minimum of 50 examples
s = dataset.groupby(['relation_label'])['column_name'].count()
no_rels = list(s[s < 50].keys())
dataset = dataset.loc[~dataset['relation_label'].isin(no_rels)]
dataset

In [ ]:
dataset['selection_type'].unique()

In [ ]:
dataset['selection_type'].replace(['Intra similarity','Inter Similarity'], 'Corner Cases', inplace=True)
dataset['selection_type'].replace(['Random', 'Random 2', 'Random 3'], 'Random', inplace=True)

In [ ]:
dataset.to_csv('output-data/cpa-datasets/dataset.csv', index=False)